# 使用Web of Science API进行文献DOI文本挖掘

## 概述
本笔记本演示如何使用Web of Science API搜索和提取学术文献数据，特别是文献的DOI信息。我们将通过以下步骤完成整个过程：

1. **第一步：定义搜索查询** - 构建适合的查询字符串
2. **第二步：配置API连接** - 设置API密钥和请求参数
3. **第三步：发送API请求** - 向WOS API发送查询请求
4. **第四步：解析JSON响应** - 从API响应中提取结构化数据
5. **第五步：数据导出** - 将提取的数据保存为Excel文件

## 第一步：定义搜索查询
首先，我们需要构建一个适合的Web of Science查询字符串。此查询专门搜索关于"阴离子交换膜"的研究文献。

### 查询字符串构建说明
- `TI=("anion exchange membrane")`: 搜索标题中包含"阴离子交换膜"的文献
- `TS=("anion exchange membrane" NOT cataly*)`: 搜索主题中包含"阴离子交换膜"但不包含催化相关词汇的文献
- `AB=("durability" OR "conductivity" OR "tensile strength" OR "swelling ratio")`: 摘要中包含耐久性、导电性、拉伸强度或溶胀比的文献
- `PY=(2020-2022)`: 限制发表年份为2020-2022年


In [8]:
query_string = """
(TI=("anion exchange membrane") OR TS=("anion exchange membrane" NOT cataly*))
AND (AB=("durability" OR "conductivity" OR "tensile strength" OR "swelling ratio"))
AND PY=(2020-2022)
"""

## 第二步：配置API连接
接下来，我们需要配置Web of Science API的连接参数，包括：
- **API密钥**：用于身份验证
- **基础URL**：WOS API的端点地址
- **请求头**：包含认证信息和接受的数据格式
- **查询参数**：包括数据库、查询字符串、限制数量、页码和排序字段

### 重要参数说明
- `db`: 数据库选择（WOS = Web of Science核心数据库）
- `q`: 查询字符串（我们在第一步中定义的）
- `limit`: 每页返回的记录数量（最大100）
- `page`: 页码（从1开始）
- `sortField`: 排序字段（LD+D = 按最新日期降序排列）

在以下链接创建账号并申请权限，获得api key
https://developer.clarivate.com/apis/wos-starter


In [ ]:
import requests
import logging

# Setup logger
logger = logging.getLogger('wos_TDM.log')
logger.setLevel(logging.INFO)

# API key and query
api_key = "<your_api_key>"
query = query_string
limit = 50
page = 1

# Base URL and headers
base_url = "https://api.clarivate.com/apis/wos-starter/v1/documents"
headers = {
    "accept": "application/json",
    "X-ApiKey": api_key
}

# Request parameters
params = {
    "db": "WOS",
    "q": query,
    "limit": limit,
    "page": page,
    "sortField": "LD+D"
}


## 第三步：发送API请求
现在我们使用配置好的参数向Web of Science API发送GET请求。代码将：
1. 使用`requests.get()`发送HTTP GET请求
2. 检查响应状态码（200表示成功）
3. 解析JSON响应数据
4. 显示总记录数和当前页面的记录数

### 状态码说明
- **200**: 请求成功
- **400**: 请求格式错误
- **401**: 认证失败（API密钥无效）
- **403**: 访问被禁止
- **429**: 请求过于频繁（超过速率限制）


In [9]:
response = requests.get(base_url, params=params, headers=headers)
if response.status_code == 200:
    logger.info(f"[WOS] Request successful. Status code: {response.status_code}")
    json_data = response.json()
    print(f"Total records: {json_data.get('metadata', {}).get('total', 'Unknown')}")
    print(f"Records in this page: {len(json_data.get('hits', []))}")
else:
    logger.error(f"[WOS] Request failed with status code {response.status_code}. Response content: {response.text}")

Total records: 535
Records in this page: 50


In [10]:
response.json()

{'metadata': {'total': 535, 'page': 1, 'limit': 50},
 'hits': [{'uid': 'WOS:001074351400001',
   'title': 'Recovery and enrichment of acid from metallurgical wastewater model by electrodialysis integrated diffusion dialysis system using poly(ethylene) based IEMs',
   'types': ['Article'],
   'sourceTypes': ['Article'],
   'source': {'sourceTitle': 'SEPARATION AND PURIFICATION TECHNOLOGY',
    'publishYear': 2023,
    'publishMonth': 'JAN 1',
    'volume': '304',
    'articleNumber': '122353',
    'pages': {'count': 9}},
   'names': {'authors': [{'displayName': 'Upadhyay, Prashant',
      'wosStandard': 'Upadhyay, P',
      'researcherId': 'EBJ-7974-2022'},
     {'displayName': 'Mishra, Sarthak',
      'wosStandard': 'Mishra, S',
      'researcherId': 'HTK-0410-2023'},
     {'displayName': 'Sharma, Jeet',
      'wosStandard': 'Sharma, J',
      'researcherId': 'AAD-2091-2022'},
     {'displayName': 'Panja, Surajit',
      'wosStandard': 'Panja, S',
      'researcherId': 'K-4301-2019'},


## 第四步：解析JSON响应
在这一步中，我们将原始的JSON响应数据转换为结构化的pandas DataFrame。这包括：

**4.1 提取基本信息**
- **UID**: 文献唯一标识符
- **Title**: 文献标题
- **Type**: 文献类型（期刊论文、会议论文等）

**4.2 提取期刊信息**
- **Journal**: 期刊名称
- **Year**: 发表年份
- **Volume**: 卷号
- **Issue**: 期号
- **Pages**: 页码范围

**4.3 提取作者信息**
- **First_Author**: 第一作者
- **Author_Count**: 作者总数
- **All_Authors**: 所有作者列表

**4.4 提取引用和标识符**
- **Citations**: 被引用次数
- **DOI**: 数字对象标识符（重要！）
- **PMID**: PubMed标识符

**4.5 提取关键词和链接**
- **Keywords**: 作者关键词
- **WOS_Link**: Web of Science记录链接


In [7]:
import pandas as pd
import json

def parse_wos_data(data):
    """
    Parse Web of Science API data into a structured DataFrame
    """
    records = []
    
    for hit in data['hits']:
        record = {}
        
        # Basic information
        record['UID'] = hit.get('uid', '')
        record['Title'] = hit.get('title', '')
        record['Type'] = ', '.join(hit.get('types', []))
        
        # Source information
        source = hit.get('source', {})
        record['Journal'] = source.get('sourceTitle', '')
        record['Year'] = source.get('publishYear', '')
        record['Volume'] = source.get('volume', '')
        record['Issue'] = source.get('issue', '')
        record['Pages'] = source.get('pages', {}).get('range', source.get('pages', {}).get('count', ''))
        
        # Authors
        authors = hit.get('names', {}).get('authors', [])
        if authors:
            record['First_Author'] = authors[0].get('displayName', '')
            record['Author_Count'] = len(authors)
            record['All_Authors'] = '; '.join([author.get('displayName', '') for author in authors])
        else:
            record['First_Author'] = ''
            record['Author_Count'] = 0
            record['All_Authors'] = ''
        
        # Citations
        citations = hit.get('citations', [])
        record['Citations'] = citations[0].get('count', 0) if citations else 0
        
        # Identifiers
        identifiers = hit.get('identifiers', {})
        record['DOI'] = identifiers.get('doi', '')
        record['PMID'] = identifiers.get('pmid', '')
        
        # Keywords
        keywords = hit.get('keywords', {}).get('authorKeywords', [])
        record['Keywords'] = '; '.join(keywords) if keywords else ''
        
        # Links
        links = hit.get('links', {})
        record['WOS_Link'] = links.get('record', '')
        
        records.append(record)
    
    return pd.DataFrame(records)

# Parse the data
df = parse_wos_data(json_data)

# Display basic info
print(f"Total records found: {json_data['metadata']['total']}")
print(f"Records in this DataFrame: {len(df)}")
print(f"\\nDataFrame shape: {df.shape}")

# Display the DataFrame
print("\\n" + "="*100)
print("RESEARCH ARTICLES TABLE")
print("="*100)
display(df.head())


Total records found: 535
Records in this DataFrame: 50
\nDataFrame shape: (50, 16)
\n====================================================================================================
RESEARCH ARTICLES TABLE


,UID,Title,Type,Journal,Year,Volume,Issue,Pages,First_Author,Author_Count,All_Authors,Citations,DOI,PMID,Keywords,WOS_Link
0,WOS:001074351400001,Recovery and enrichment of acid from metallurg...,Article,SEPARATION AND PURIFICATION TECHNOLOGY,2023,304,,9,"Upadhyay, Prashant",5,"Upadhyay, Prashant; Mishra, Sarthak; Sharma, J...",13,10.1016/j.seppur.2022.122353,,Diffusion dialysis; Electrodialysis; Acid reco...,https://www.webofscience.com/api/gateway?GWVer...
1,WOS:001009928800001,State-of-the-art and developmental trends in p...,Review,APPLIED CATALYSIS B-ENVIRONMENT AND ENERGY,2023,325,,38,"Hossen, Mosaddek",7,"Hossen, Mosaddek; Hasan, Shamim; Sardar, Riaju...",131,10.1016/j.apcatb.2022.121733,,Anion; alkaline exchange membrane fuel cell; A...,https://www.webofscience.com/api/gateway?GWVer...
2,WOS:000967455200001,High-performance anion exchange membranes achi...,Article,JOURNAL OF MEMBRANE SCIENCE,2022,664,,11,"Min, Kyungwhan",5,"Min, Kyungwhan; Lee, Yerim; Choi, Yeongeun; Kw...",41,10.1016/j.memsci.2022.121071,,Anion exchange membranes; Aryl-ether free poly...,https://www.webofscience.com/api/gateway?GWVer...
3,WOS:000964108100001,Towards high alkaline stability and fuel cell ...,Article,JOURNAL OF POWER SOURCES,2023,557,,10,"Wang, Tao",7,"Wang, Tao; Zhao, Yu; Wang, Sheng; Cheng, Sheng...",31,10.1016/j.jpowsour.2022.232590,,Poly(arylene alkylene)s; Anion exchange membra...,https://www.webofscience.com/api/gateway?GWVer...
4,WOS:000933902300005,Probing the effect of ionomer swelling on the ...,Article,JOURNAL OF POWER SOURCES,2022,550,,10,"Chen, Binyu",3,"Chen, Binyu; Mardle, Peter; Holdcroft, Steven",47,10.1016/j.jpowsour.2022.232134,,Anion exchange membrane; Water electrolysis; S...,https://www.webofscience.com/api/gateway?GWVer...


## 第五步：数据导出


In [23]:
# save df to excel
df.to_excel('wos_article_metadata.xlsx', index=False)

### 下一步
有了以上文献的元数据信息，可以进行简单的筛选和分析。
1. **筛选文献**: 可以根据关键词、作者或年份来筛选文献。
2. **分析趋势**: 分析不同年份的研究趋势。
3. **链接生成**: 使用DOI生成可访问链接（例如：https://doi.org/10.1016/j.jpowsour.2022.232134 ）。

**全文获取路径**

针对不同需求，全文的获取路径也不同：

- 若仅需摘要：可调用 Web of Science 的 Expand API 或 Crossref 的 API 来获取摘要信息。

- 若需要全文：
    - API 接口：推荐使用 Elsevier 等出版商提供的官方 API 或文本挖掘 (Text Mining) 接口，这是合规且高效的方式。
    - 网页抓取：可编写爬虫程序从网页直接获取 HTML 格式的全文，但操作时必须严格遵守目标平台的服务条款和文本挖掘规定。
    - 备用方案：Sci-Hub 可作为获取 PDF 全文的补充途径，但其数据通常更新至2022年之前。